## Este codigo está diseñado para que copie la informacion publica de las empresas, asimismo tambien que copie las mixtas borrando la informacion confidencial.
### Nota: Click en Run All

In [ ]:
import pandas as pd
import os
from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

import openpyxl
import shutil
import re
import time

In [ ]:
trimestre = input('inserte el trimestre en numeros (1 2 3 4)')
if trimestre in ['1','2','3']:
    formatos = ["1","2","4","5","6","13","14","16","17","18","19","20","21","25","26","27","29","30","31","33","34","36","37","43","44","45","46","47","48","49","50","51","54","55","56","57","59","61","63","64","65","66","67","68","72","83","84","85","86","87","91","95","96","97","98","99","100"]
elif trimestre == '4':
    formatos = ["1","2","4","5","6","13","14","16","17","18","19","20","21","25","26","27","29","30","31","33","34","36","37","43","44","45","46","47","48","49","50","51","54","55","56","57","59","61","63","64","65","66","67","68","72","83","84","85","86","87",'88','89','90',"91",'92','93',"95","96","97","98","99","100"]
else:
    print('error en el trimestre')

In [ ]:
formatos = [str(x).zfill(3) for x in formatos]

In [ ]:

# Rutas de origen y destino
ruta_origen = input("Ingresa la ruta de origen: ")
ruta_origen = ruta_origen.replace('\\', '\\\\')

ruta_destino = input("Ingresa la ruta de destino: ")
ruta_destino = ruta_destino.replace('\\', '\\\\')

# Obtener la lista de carpetas de empresas
carpetas_empresas = [nombre for nombre in os.listdir(ruta_origen) if os.path.isdir(os.path.join(ruta_origen, nombre))]

# Ingreso de la fecha mínima de modificación
fecha_minima = input('inserte la fecha minima de modificación de los archivos:%d/%m/%Y')
try:
    fecha_minima = time.mktime(time.strptime(fecha_minima, "%d/%m/%Y"))
except ValueError:
    print("Fecha mínima inválida. Asegúrate de usar el formato DD/MM/AAAA.")
    exit()

# Ingreso de la fecha máxima de modificación
fecha_maxima = input("Inserte la fecha maxima de modificacion, %d/%m/%Y")
try:
    fecha_maxima = time.mktime(time.strptime(fecha_maxima, "%d/%m/%Y"))
except ValueError:
    print("Fecha máxima inválida. Asegúrate de usar el formato DD/MM/AAAA.")
    exit()

# Recorrer cada carpeta de empresa
for carpeta_empresa in carpetas_empresas:
    ruta_empresa = os.path.join(ruta_origen, carpeta_empresa)

    archivos_por_formato = {}  # Diccionario para almacenar los archivos por formato

    # Obtener la lista de archivos que cumplen con los formatos especificados y el rango de fechas
    for formato in formatos:
        archivos = [
            nombre for nombre in os.listdir(ruta_empresa)
            if nombre.startswith(formato) and nombre.lower().endswith('xlsx') and fecha_minima <= os.path.getmtime(os.path.join(ruta_empresa, nombre)) <= fecha_maxima
        ]

        if archivos:
            archivos_por_formato[formato] = archivos

    # Verificar si se encontraron archivos
    if archivos_por_formato:
        # Crear la carpeta de la empresa en la ruta de destino
        ruta_destino_empresa = os.path.join(ruta_destino, carpeta_empresa)
        os.makedirs(ruta_destino_empresa, exist_ok=True)

        # Copiar los archivos más recientes de cada formato a la carpeta de destino
        for formato, archivos in archivos_por_formato.items():
            archivo_mas_reciente = max(archivos, key=lambda x: os.path.getmtime(os.path.join(ruta_empresa, x)))
            ruta_origen_archivo = os.path.join(ruta_empresa, archivo_mas_reciente)
            ruta_destino_archivo = os.path.join(ruta_destino_empresa, archivo_mas_reciente)
            shutil.copy2(ruta_origen_archivo, ruta_destino_archivo)

        print(f"Se copiaron los archivos más recientes de los formatos {', '.join(archivos_por_formato.keys())} de la empresa {carpeta_empresa}.")

print("Proceso finalizado.")

In [ ]:
# Obtener el directorio
wd = ruta_destino
# Columnas a borrar para cada tipo de archivo
col_borrar = {
    '043': ['Número total de Altas', 'Número total de Bajas'],
    '045': ['Número total de Altas', 'Número total de Bajas'],
    '054': ['Nombre y apellido del abonado', 'Número del servicio publico móvil objeto del cuestionamiento'],
    '055': ['Nombre y apellido del abonado', 'Número del servicio publico móvil objeto del cuestionamiento'],
    '056': ['Tráfico', 'Ingresos'],
    '057': ['Tráfico', 'Ingresos'],
    '063': ['Tráfico', 'Ingresos'],
    '059': ['Capacidad E1', 'Tráfico', 'Ingresos'],
    '061': ['Recibos', 'Ingresos'],
    '064': ['Cantidad'],
    '085': ['Tarifa mensual del enlace (dólares)']
}

# Recorrer todas las carpetas y subcarpetas dentro del directorio principal
for ruta, carpetas, archivos in os.walk(wd):
    for archivo in archivos:
        archivo_path = os.path.join(ruta, archivo)

        # Obtener el tipo de archivo basado en el número inicial
        tipo_archivo = archivo[:3]

        if tipo_archivo in col_borrar and archivo.endswith('.xlsx'):
            print(archivo)
            book = load_workbook(archivo_path)

            # Recorrer cada hoja de cálculo del archivo
            for hoja_calculo in book.sheetnames:
                sheet = book[hoja_calculo]

                # Buscar las columnas a borrar
                columnas_borrar = []
                for columna in sheet.iter_cols():
                    for celda in columna:
                        if celda.value in col_borrar[tipo_archivo]:
                            columnas_borrar.append(celda.column_letter)

                # Borrar los datos debajo de las columnas encontradas
                # Dentro del bucle que recorre las celdas
                for columna_modalidad in columnas_borrar:
                    for fila in range(22, sheet.max_row + 1):
                        celda = f'{columna_modalidad}{fila}'
                        
                        # Verificar si la celda está dentro de algún rango de celdas fusionadas
                        is_merged = False
                        merged_range_to_remove = None
                        for merged_range in sheet.merged_cells.ranges:
                            if celda in merged_range:
                                is_merged = True
                                merged_range_to_remove = merged_range
                                break
                        
                        # Si la celda está fusionada, deshacer la fusión
                        if is_merged:
                            # Deshacer la fusión del rango específico
                            sheet.unmerge_cells(str(merged_range_to_remove))
                        
                        # Establecer el valor en la celda
                        sheet[celda].value = None

            book.save(archivo_path)

## PARA FINANCIEROS

In [ ]:
borrar_rango = ['C23:D40','C42:D46','C48:D49']

In [ ]:
borrar = 'C23:D40'

In [ ]:
import os
import openpyxl
from openpyxl.utils import get_column_letter

# Carpeta donde se encuentran los archivos Excel
carpeta = r'C:\Users\colazabal\Desktop\prueba de copia formatos enteros'

# Rango a borrar
borrar_rango = 'C23:D40'

# Recorremos los archivos en la carpeta
for rutas, carpetas, archivos in os.walk(carpeta):
    for archivo in archivos:
        if archivo.startswith('091') and archivo.lower().endswith('.xlsx'):
            # Abre el archivo Excel
            wb = openpyxl.load_workbook(os.path.join(rutas, archivo))
            
            # Accede a la hoja de trabajo (puede cambiar el nombre de la hoja según tu caso)
            sheet = wb.active

            # Lista de rangos de celdas que deseas borrar
            rangos_a_borrar = sheet[borrar_rango]

            # Obtener las dimensiones de la hoja de cálculo
            max_row = sheet.max_row
            max_col = sheet.max_column

            # Iterar sobre las celdas para evaluar y reemplazar las fórmulas con resultados
            for row in range(1, max_row + 1):
                for col in range(1, max_col + 1):
                    cell = sheet.cell(row=row, column=col)
                    if cell.data_type == 'f':  # Verificar si la celda contiene una fórmula
                        try:
                            # Evaluar la fórmula y obtener el resultado
                            resultado = cell.value
                            cell.value = resultado  # Reemplazar la fórmula con el resultado
                        except Exception as e:
                            print(f"Error al evaluar la fórmula en la celda {get_column_letter(col)}{row}: {str(e)}")

            # Iterar sobre los rangos y borrar las celdas en cada uno de ellos
            for rango in rangos_a_borrar:
                for cell in rango:
                    cell.value = None

            # Guardar los cambios en el archivo
            wb.save(os.path.join(rutas, archivo))
            wb.close()


In [ ]:
#CODIGO PARA BORRAR DATOS DEL 091
# Lista de nombres de archivos
rangos_a_borrar = ["A1:B10", "C3:E15", "F1:G8"]
for rutas, carpetas, archivos in os.walk(carpeta):
    for archivo in archivos:
        if archivo.startswith('091') and archivo.lower().endswith('.xlsx'):
            wb = openpyxl.load_workbook(archivo)
            
            for hoja in wb.sheetnames:
                # Selecciona la hoja de trabajo
                worksheet = wb[hoja]
                
                # Recorre todas las celdas con fórmulas y convierte las fórmulas en valores
                for fila in worksheet.iter_rows(values_only=True):
                    for celda in fila:
                        if isinstance(celda, str) and celda.startswith("="):
                            celda.value = None  # Borra las fórmulas, dejando las celdas en blanco si eran fórmulas
            for rango in rangos_a_borrar:
                # Selecciona la hoja de trabajo (puedes cambiar el nombre de la hoja si es necesario)
                hoja = wb.active
                # Borra el rango de celdas
                hoja[rango] = None
            
            # Guarda los cambios
            wb.save(archivo)
            # Cierra el archivo
            wb.close()